In [1]:
# データ分析・操作用ライブラリ
import numpy as np
import pandas as pd
from collections import Counter
import itertools

# NLP用ライブラリ
import MeCab,re
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
# CSVの読み込み
wikiData = pd.read_csv("csv/train.csv", encoding="utf-8")

# 読み込みデータの表示
wikiData.head()

,text,target
0,ドール・フード・カンパニー ドール・フード・カンパニー（Dole Food Compan...,1
1,南ヶ丘牧場 株式会社 南ヶ丘牧場（みなみがおかぼくじょう）は、栃木県那須高原に本拠を置く...,1
2,岩瀬牧場 岩瀬牧場（いわせぼくじょう）は、日本の福島県岩瀬郡鏡石町にある牧場である。 ...,1
3,"クラギ クラギ株式会社（くらぎ、""KURAGI Co.Ltd."" ）は、三重県を中心に「...",1
4,"ハッピーネモファーム 株式会社ハッピーネモファーム（""""）は、北海道浦河郡浦河町にある競...",1


In [3]:
# 形態素解析用関数
def tokenize(text):
    tokens = []
    tagger = MeCab.Tagger( "-Ochasen" )
    node = tagger.parseToNode(text)
    while node:
        if node.feature.split(",")[0] == "名詞":
                replace_node = re.sub( re.compile( "[!-/:-@[-`{-~]" ), "", node.surface )
                if replace_node != "" and replace_node != " ":
                    tokens.append(replace_node)
        node = node.next
    return tokens

In [4]:
# ストップワードの除去
path = "data/Slothlib.txt"
stopWords = []
with open(path, encoding="utf-8") as f:
    stopWords = [s.strip() for s in f.readlines()]
    print(stopWords)

cleanedData = wikiData.copy()

for word in stopWords:
    cleanedData['text'] = cleanedData['text'].str.replace(word,'')

['あそこ', 'あたり', 'あちら', 'あっち', 'あと', 'あな', 'あなた', 'あれ', 'いくつ', 'いつ', 'いま', 'いや', 'いろいろ', 'うち', 'おおまか', 'おまえ', 'おれ', 'がい', 'かく', 'かたち', 'かやの', 'から', 'がら', 'きた', 'くせ', 'ここ', 'こっち', 'こと', 'ごと', 'こちら', 'ごっちゃ', 'これ', 'これら', 'ごろ', 'さまざま', 'さらい', 'さん', 'しかた', 'しよう', 'すか', 'ずつ', 'すね', 'すべて', 'ぜんぶ', 'そう', 'そこ', 'そちら', 'そっち', 'そで', 'それ', 'それぞれ', 'それなり', 'たくさん', 'たち', 'たび', 'ため', 'だめ', 'ちゃ', 'ちゃん', 'てん', 'とおり', 'とき', 'どこ', 'どこか', 'ところ', 'どちら', 'どっか', 'どっち', 'どれ', 'なか', 'なかば', 'なに', 'など', 'なん', 'はじめ', 'はず', 'はるか', 'ひと', 'ひとつ', 'ふく', 'ぶり', 'べつ', 'へん', 'ぺん', 'ほう', 'ほか', 'まさ', 'まし', 'まとも', 'まま', 'みたい', 'みつ', 'みなさん', 'みんな', 'もと', 'もの', 'もん', 'やつ', 'よう', 'よそ', 'わけ', 'わたし', '', 'ハイ', '', '', '上', '中', '下', '字', '', '', '年', '月', '日', '時', '分', '秒', '週', '火', '水', '木', '金', '土', '国', '都', '道', '府', '県', '市', '区', '町', '村', '', '', '各', '第', '方', '何', '的', '度', '文', '者', '性', '体', '人', '他', '今', '部', '課', '係', '外', '類', '達', '気', '室', '口', '誰', '用', '界', '会', '首', '男', '女', '別', '話', '私', '屋', '店', '家', '場'

In [5]:
cleanedData.head()

,text,target
0,ドール・フード・カンパニー ドール・フード・カンパニー（Dole Food Compan...,1
1,南ヶ丘牧 株社 南ヶ丘牧（みなみがおかぼくじょう）は、栃那須原に本拠を置く酪農経営企業。...,1
2,岩瀬牧 岩瀬牧（いわせぼくじょう）は、本の福島岩瀬郡鏡石にある牧である。 鏡石のシンボ...,1
3,"クラギ クラギ株社（くらぎ、""KURAGI Co.Ltd."" ）は、重を心に「農業」の舗...",1
4,"ハッピーネモファーム 株社ハッピーネモファーム（""""）は、北海浦河郡浦河にある競走馬（サ...",1


In [6]:
# Numpy配列へ変換
X = cleanedData.text.values
y = cleanedData.target.values

In [7]:
# 形態素解析
words = []
for text in X:
    words.append(tokenize(text))

In [8]:
# 1次元配列へ変換
words_1d = itertools.chain.from_iterable(words)

In [9]:
counter = Counter(words_1d)
for word, count in counter.most_common():
    print(f"{word}: {count}")

社: 57300
株: 26376
業: 19664
販売: 12059
の: 11345
本社: 10935
本: 10488
舗: 10443
平成: 9957
企業: 9907
2: 9283
1: 9111
東京: 8939
製造: 8772
製: 8339
現在: 8053
グループ: 7879
バス: 7876
3: 7532
電: 7099
設立: 7066
商: 6989
開発: 6558
放送: 6204
展開: 5960
昭和: 5922
以: 5797
機: 5695
営業: 5430
メーカー: 5354
代: 5337
関: 5320
工: 5214
経営: 5181
運営: 5145
駅: 5046
銀: 4889
大阪: 4723
4: 4683
心: 4660
路: 4583
ブランド: 4529
変更: 4525
開始: 4508
創業: 4436
期: 4237
同社: 4148
主: 4145
サービス: 4038
車: 4023
運: 3989
5: 3955
位: 3868
戦: 3819
制: 3785
使: 3784
合併: 3754
鉄: 3742
食: 3609
業務: 3461
自動車: 3317
称: 3304
同: 3246
社長: 3157
向け: 3102
局: 3037
）: 2986
テビ: 2975
生産: 2955
発表: 2860
工業: 2836
世: 2813
10: 2800
番組: 2795
域: 2748
門: 2742
野: 2739
6: 2689
報: 2637
システム: 2585
発売: 2563
カード: 2553
提供: 2543
号: 2501
車両: 2492
導入: 2474
7: 2445
北: 2429
8: 2388
技術: 2349
山: 2335
央: 2329
売: 2321
証券: 2296
型: 2289
取引: 2257
合: 2252
統合: 2248
提携: 2211
学: 2179
海: 2177
京: 2170
産: 2167
CM: 2163
田: 2148
存在: 2140
当初: 2137
州: 2136
ホールディングス: 2125
9: 2105
階: 2100
完全: 2060
長: 2060
代表: 2056
リーグ: 2053

フランス: 195
武田: 195
竣工: 195
コジマ: 195
小田急: 194
きょう: 194
63: 194
s: 194
56: 194
里: 194
受信: 194
天然: 193
激: 193
キャンペーン: 193
53: 193
ページ: 193
太平洋: 192
ドラマ: 192
バー: 192
任: 192
お客: 191
プラザ: 191
スカイ: 191
絡: 191
崩壊: 191
酸: 191
インターナショナル: 191
セール: 191
支局: 191
JFL: 191
改: 190
長岡: 190
1948: 190
審査: 190
在庫: 190
平和: 190
1953: 190
イタリア: 190
ステーション: 190
稼働: 189
不況: 189
模: 189
1966: 189
桜: 188
ライフ: 188
相当: 188
記述: 188
マシン: 188
セキュリティ: 188
署: 187
1952: 187
パネル: 187
小林: 187
伊: 187
ラインナップ: 187
畑: 187
乗り: 187
プラスチック: 186
入社: 186
アルミ: 186
袋: 186
拓: 186
トン: 185
要請: 185
46: 185
発注: 185
プラス: 185
動き: 185
般路: 185
塾: 185
周波: 185
電波: 185
牛乳: 184
パイオニア: 184
リゾート: 184
帳: 184
ソース: 184
紋: 184
福: 184
乳: 183
山本: 183
91: 183
パーツ: 183
セクター: 183
音声: 183
扉: 183
光学: 183
ランキング: 182
1959: 182
交流: 182
済み: 181
121: 181
運動: 181
1954: 181
認可: 181
炉: 181
領域: 181
表現: 181
争い: 181
ボール: 181
当該: 181
57: 181
リテール: 181
網走: 181
改良: 180
振興: 180
59: 180
密着: 180
重要: 180
アップ: 180
1946: 179
流れ: 179
バンク: 179
断念: 179
状: 179
災: 179
頭取: 179
プリンス: 179

更: 87
不可能: 87
発車: 87
刷新: 87
虎: 87
展: 87
スタンダード: 87
SA: 87
祝: 87
プラモデル: 87
マスター: 87
授業: 87
為替: 87
冊: 87
クリスタル: 87
富重: 87
ワンステップ: 87
ウエルシア: 87
工房: 86
メリット: 86
安心: 86
20063: 86
1500: 86
はつ: 86
1937: 86
白石: 86
トップクラス: 86
リフォーム: 86
美術館: 86
ウエスト: 86
清掃: 86
勧業: 86
着工: 86
参: 86
マルチ: 86
ヤクルト: 86
君: 86
粉: 86
テン: 86
黄: 86
緩和: 86
アイ・ホールディングス: 86
加速: 86
沼: 86
SUPER: 86
講座: 86
ベイ: 86
センサ: 86
余: 85
らい: 85
マイ: 85
山山: 85
1936: 85
早期: 85
長谷川: 85
20134: 85
20094: 85
指揮: 85
主軸: 85
はく: 85
伊那: 85
ファイナンス: 85
穴: 85
死: 85
ヘッド: 85
実証: 85
井筒: 85
鋼材: 85
ルノー: 85
古河電工: 85
互換: 85
訳: 84
空調: 84
77: 84
依存: 84
試: 84
セールス: 84
教授: 84
官: 84
京セラ: 84
接着: 84
始まり: 84
評判: 84
給食: 84
独特: 84
要: 84
草津: 84
鮮魚: 84
1919: 84
攻撃: 84
鈴鹿: 84
隔: 84
深川: 84
800: 84
空輸: 84
同局: 84
洗濯: 84
SAP: 84
死亡: 83
青梅: 83
増強: 83
デベロッパー: 83
審: 83
梅: 83
石井: 83
淀川: 83
ペイント: 83
能登: 83
辻: 83
出展: 83
2020: 83
声優: 83
宣言: 83
経過: 83
満足: 83
タイアップ: 83
任天堂: 83
20174: 83
削除: 83
早朝: 83
20154: 83
201612: 83
補完: 83
HDD: 83
教材: 83
ニッケル: 83
京浜: 83
エイト: 83
ダイキ: 83
TSUTAYA: 83

ヤフー: 51
営鉄: 51
衣装: 51
フランス語: 51
講演: 51
実践: 51
スギ: 51
柔軟: 51
バックアップ: 51
本選: 51
受講: 51
ジーンズ: 51
リチウム: 51
ステオ: 51
Technology: 51
フォーマット: 51
非鉄: 51
パーソナルコンピュータ: 51
駿: 51
ウィルコム: 51
Mbps: 51
遊覧: 51
MasterCard: 51
貨協: 51
マルエツ: 51
ミドリ: 51
プー: 51
是正: 50
背: 50
菌: 50
断熱: 50
尾張: 50
201712: 50
ミサワホーム: 50
オフィシャルスポンサー: 50
831: 50
弁済: 50
アルファ: 50
半田: 50
神宮: 50
殺到: 50
お茶: 50
正宗: 50
がち: 50
カプセル: 50
養: 50
和泉: 50
栗原: 50
変遷: 50
貯蓄: 50
クラフト: 50
原型: 50
横山: 50
アルバム: 50
監修: 50
笠: 50
バランス: 50
テビドラマ: 50
マック: 50
メーカ: 50
圭: 50
113: 50
20111: 50
クラウン: 50
賀: 50
勢い: 50
安曇: 50
商売: 50
剣: 50
活発: 50
騒動: 50
スーツ: 50
収納: 50
舗以: 50
くすり: 50
カセット: 50
音波: 50
IHI: 50
サティ: 50
ストロボ: 50
アンプ: 50
コロムビア: 50
JOYSOUND: 50
精算: 50
ウイング: 50
Bank: 50
MEGA: 50
運輸省: 50
ジュンク: 50
MTM: 50
勘定: 50
BBC: 50
フィナンシャル・グループ: 50
親和: 50
西濃運輸: 50
カリフォルニア: 49
インク: 49
原油: 49
陶磁: 49
徳山: 49
垣: 49
挙: 49
123: 49
大倉: 49
知多: 49
詐欺: 49
寸: 49
新川: 49
コン: 49
マスク: 49
ょ: 49
物販: 49
20186: 49
ボトラー: 49
恵: 49
20166: 49
ぞう: 49
くり: 49
披露: 49
物理: 49
ミッション: 49
小島: 49
20099: 49


休眠: 35
コークス: 35
ホ: 35
申し込み: 35
ステッカー: 35
舗態: 35
ーシングカー: 35
双葉: 35
転入: 35
騒音: 35
ボンネット: 35
虎彦: 35
エコー: 35
mobile: 35
カーナビゲーション: 35
編入: 35
タッチ: 35
ダイハツ工業: 35
拓也: 35
UAZ: 35
清原: 35
スクウェア: 35
ニッショーストア: 35
乗り継ぎ: 35
PG: 35
WACK: 35
南米: 34
1905: 34
扇: 34
1125: 34
ドラゴンズ: 34
開港: 34
了: 34
ナカ: 34
漏れ: 34
永田: 34
警視庁: 34
偽造: 34
社東: 34
ゅう: 34
桟橋: 34
彦根: 34
ロケ: 34
合板: 34
斜: 34
全焼: 34
突入: 34
製糸: 34
20165: 34
20092: 34
ウッド: 34
シード: 34
熱田: 34
麹: 34
農産物: 34
害: 34
富岡: 34
燃: 34
熱海: 34
廿: 34
匠: 34
ナビスコ: 34
～」: 34
寿し: 34
オリオン: 34
楠: 34
ロアッソ: 34
1916: 34
麻: 34
薩摩: 34
意味合い: 34
デッキ: 34
せんべい: 34
客層: 34
強豪: 34
20142: 34
桜井: 34
筑波: 34
20033: 34
かなり: 34
斎藤: 34
AIR: 34
まき: 34
LS: 34
紛争: 34
20112: 34
ウルトラ: 34
ドーナツ: 34
TVQ: 34
禁煙: 34
なり: 34
リコール: 34
ロット: 34
想: 34
山科: 34
荒: 34
スマイル: 34
みな: 34
20067: 34
321: 34
となり: 34
Tシャツ: 34
自由が丘: 34
悦: 34
200411: 34
選考: 34
染: 34
坊: 34
広範囲: 34
312: 34
ブロンコス: 34
優遇: 34
20075: 34
搬入: 34
20003: 34
紡: 34
SPA: 34
教師: 34
ループ: 34
730: 34
寝具: 34
本件: 34
梱包: 34
メイク: 34
絵具: 34
鉛: 34
20145: 34
納期

...

ウォーカープラス・ドット・コム: 1
MovieWalker: 1
WeddingWalker: 1
TravelWalker: 1
SalonWalker: 1
RoomWalker: 1
ウォーカーオンラインストア: 1
200032: 1
グランジェ: 1
BIGRUN: 1
20092011: 1
ミッセ: 1
セミナーツ: 1
BiZ: 1
200933: 1
オリコン・エンタテインメント: 1
2015103012: 1
信ぴょう: 1
SLAPP: 1
ミヤテ: 1
JOMM: 1
心霊: 1
1948725: 1
1955211: 1
デイリースポーツプスセンター: 1
デイリースポーツ・クオリティ: 1
びるで: 1
巨辺: 1
巨以: 1
彩佳: 1
1967121: 1
専荷: 1
窟賞: 1
夜祭: 1
198363: 1
ハルク・ホーガン: 1
ホーガン: 1
アックスボンバー: 1
激写: 1
プロスマニア: 1
寸評: 1
ジュニアフライ: 1
2006820071: 1
毒物: 1
エヌシーアール: 1
宮堂: 1
ホストコンピュータシステム: 1
エイ・ティ・アンド・ティ: 1
エヌ・シー・アールホールディングス: 1
グリザイアシリーズ: 1
JOWY: 1
不偏: 1
ビショップ: 1
2010724: 1
ぎいせはら: 1
Atsugi: 1
Isehara: 1
Cablenetwork: 1
セクターケーブルテビ: 1
34758: 1
ダビング: 1
ぎっ: 1
saQwa: 1
イオンダイクト: 1
TenLtd: 1
トリノパラリンピックメダリスト: 1
深雪: 1
ソルトイクシティパラリンピック: 1
オラクルコーポーション: 1
199925: 1
2000428: 1
カスタマー・エクスペリエンス: 1
OpenWorld: 1
Exadata: 1
Exalogic: 1
Exalytics: 1
SPARC: 1
プロセッサー: 1
Solaris: 1
SuperCluster: 1
ニューオータニガーデンコート: 1
オールド・イングリッシュ・シープドッグ: 1
「！」: 1
イングリッシュ・シープドッグ: 1
ゅうかんじゅうたくしんぶんしゃ: 1
1985615: 1
徳康: 1
ミラマックス: 1


kW×: 1
ょうりんかい: 1
防波堤: 1
ビフェニル: 1
椎柴: 1
エネルコン: 1
アグリゲータ: 1
Liquefied: 1
9543: 1
荒田島: 1
函南: 1
経済企画庁: 1
2018117: 1
和智: 1
クローヴィアカード: 1
エネックス: 1
2014312: 1
ハンファグループ: 1
ELECTRICITY: 1
イオンフックスグループ: 1
運開: 1
山奥: 1
霞沢: 1
安房峠: 1
54410: 1
新々: 1
1268: 1
20253: 1
17548: 1
21423: 1
側島: 1
11786: 1
橋川: 1
2910: 1
20775: 1
269703: 1
5830: 1
12433: 1
65946: 1
160560: 1
4913: 1
3061: 1
づでんりょく: 1
猪苗代湖: 1
阿賀川: 1
飯豊: 1
自給自足: 1
2540: 1
loop: 1
16699: 1
ミツウロコグリーンエネルギー: 1
ょうどうどうりょく: 1
16766: 1
OWGS: 1
受槽: 1
ょうどうでんりょく: 1
ゅうおうせんとらるがす: 1
花垣: 1
在田: 1
与板: 1
254854: 1
84103: 1
25575: 1
7989: 1
29245: 1
1082: 1
404487: 1
弥彦: 1
関ケ原: 1
関配: 1
3657: 1
19441400: 1
893: 1
1913800: 1
ゅや: 1
200489: 1
199129: 1
ECCS: 1
警浪: 1
酬返: 1
浜原: 1
1316: 1
197218: 1
藤里: 1
7957405: 1
1647655: 1
4065210: 1
247400: 1
JCO: 1
いただき: 1
テプコ: 1
TEPCO: 1
セプコ: 1
CEPCO: 1
エプコ: 1
トエプコ: 1
TOEPCO: 1
エネプコ: 1
ENEPCO: 1
EASTERN: 1
ohoku: 1
enryoku: 1
雨冠: 1
シナジアパワー: 1
1641: 1
18175450: 1
11681150: 1
3274000: 1
228600: 1
319300: 1
ゅうぶでんりょく: 1
energia: 1
33138

フデリク・セペダ: 1
テキサス・ンジャーズマイルズ・マイコラス: 1
ポダ: 1
ホアン: 1
アックス・カステヤーノス: 1
カステヤーノス: 1
マイコラス・: 1
貧打: 1
ジャイアンツ・ファンフェスタ: 1
アルキメデス: 1
ちぐはぐ: 1
誤算: 1
セリーグ: 1
岩橋: 1
アックス・ゲーロ: 1
老川: 1
アダメス: 1
脇腹: 1
涼介: 1
焼け石: 1
廖任: 1
磊: 1
アルキメデス・カミネロ: 1
エキシビジョンゲーム: 1
パドスクリスチャン・ビヤヌエバ: 1
マリナーズライアン・クック: 1
岩隈: 1
いっぽう: 1
除幕: 1
1936925: 1
193751: 1
194076: 1
1950628: 1
ょじん: 1
GLOVE: 1
オンジユニフォーム: 1
ジャイアンツオフィシャルストア: 1
ヤクルトスワローズ・: 1
GSDB: 1
チアチーム・マスコットキャラクター: 1
196197: 1
コーチスボックス: 1
秀之助: 1
球審: 1
ラインアウト: 1
2018920: 1
003: 1
塁審: 1
晴信: 1
聡志: 1
原沢: 1
染髪: 1
入来: 1
門奈: 1
コトー: 1
グラッデン: 1
コーンロウ: 1
短髪: 1
アンダーシャツ: 1
克則: 1
ジー・ジー: 1
MICHEAL: 1
19501: 1
ビジターユニフォーム: 1
永沢: 1
林友: 1
なまこ: 1
1960611: 1
ホーム・ビジター: 1
ドーラン: 1
代弁: 1
クリーブランド・インディアンス: 1
ミルウォーキー・ブーブス: 1
御免: 1
未到: 1
スターズ: 1
京太: 1
札幌山: 1
スタルヒン: 1
2012725: 1
800400: 1
1959625: 1
併継: 1
テクニカルラボ: 1
モバイルテクニカルセンター: 1
Prestige: 1
プロパティアシスト: 1
キャッシュス・メディカルサービス: 1
ワランティ: 1
Aranmare: 1
アランマー: 1
プローサー: 1
UKYO: 1
クリテリウム: 1
ウィーラースクールジャパン: 1
ling: 1
アップビーリングシステム: 1
ブリッツェン・ステラ: 1
幅跳び: 1
シャトルランリー: 1
チームプイ: 

DOOR: 1
SPORESH: 1
ZIMA: 1
ロンドンフルー: 1
ポルシェ・カラカップ: 1
主浩: 1
フッシュエンジェルズ: 1
フエン: 1
佐崎: 1
新芽: 1
メリサ: 1
プラネッツ: 1
POTENCIA: 1
パチンコ・パチスロホール: 1
REITO: 1
SENSE: 1
20190209: 1
PERSONNE: 1
ペルソンアンドペルソンエンターテインメント: 1
Personne: 1
ファイナンシャルプランナー: 1
アウトソーシング・コンサルティング: 1
ENPRO: 1
エンタテイメント・プロダクツ: 1
SEIRYO: 1
湖山: 1
エヌケーシーグループ: 1
SHINNIHON: 1
1971121: 1
SaiProduction: 1
JADMA: 1
ロッピング: 1
Ropp: 1
ng: 1
Kalafina: 1
KOTOKO: 1
キャプテンハーロック: 1
メーテル: 1
鉄郎: 1
Fabbit: 1
アイドルサブカルチャー: 1
サイバークリック: 1
2081: 1
アメーバピグ: 1
OPENREC: 1
FRESH: 1
ソーシャルメディアサービス: 1
グランブルーファンタジー: 1
Cygames: 1
20181017: 1
CAAnimation: 1
シーエーアニメーション: 1
ステルスマーケティング: 1
Spotlight: 1
スポットライト: 1
リアルカイジ: 1
Name: 1
HERBE: 1
エリートモデルルック: 1
ギグマネジメントジャパン: 1
elite: 1
MODELS: 1
イヤーズコンサルティング: 1
イヤーズ・コンサルティング: 1
スギノマシン: 1
長野原: 1
726000: 1
久世殿城: 1
マルビル: 1
クオンタムリープ・: 1
クオンタム・エンターテイメント: 1
2007422: 1
クオンタムリープ: 1
南笠: 1
諸派: 1
大過: 1
乃雁: 1
静代: 1
川竹: 1
マーガット・ユキ: 1
利英: 1
姫宮: 1
タント・ミス・バージニア: 1
キドシン: 1
和助: 1
兵助: 1
バラエティ・ショー: 1
昌久: 1
済々: 1
切抜き: 1
ピー・シー・エル: 1
ミスワカナ・: 1
ミルク・ブラザース: 

フアン・マヌエル・リージョ: 1
大番: 1
リージョ: 1
ダビド・ビジャ: 1
ヴィトーリア・セトゥーバル: 1
ダンクー: 1
セルジ・サンペール: 1
アフメド・ヤセル: 1
プシーズンツアー: 1
吹東: 1
テクパーク: 1
フットボールクラブユース・ジュニアユース: 1
Vitamin: 1
ヴァイタミンクラブ: 1
福谷: 1
寮長: 1
寮母: 1
consadole: 1
ole: 1
眼光: 1
ブリザード: 1
ドー: 1
2014129: 1
コクションハウス: 1
オフィシャルダンスドリルチーム: 1
1997515: 1
ミスダンスドリルチーム: 1
摩季: 1
リーグヨゼフ: 1
ルボシュ: 1
19871996: 1
ウーゴ・フェルナンデス: 1
新明: 1
パベル: 1
鍾成: 1
ウーゴ・マラドーナ: 1
ハーフナー・ディド: 1
原新: 1
亜細亜大学: 1
黄川田: 1
賢司: 1
1996613: 1
1997102930: 1
不敗: 1
英章: 1
徳明: 1
敦彦: 1
以失: 1
19961998: 1
貴光: 1
ディド: 1
ロナウジーニョ: 1
コパ・アメリカ: 1
小松崎: 1
ロシアサッカー・プミアリーグ: 1
スパルタク・モスクワブラジルロブソン: 1
鳴り物入り: 1
マクサンドロ: 1
コーディネータ: 1
イバンチェビッチ・ラドミロ: 1
ユーゴスラビア: 1
バーヤック: 1
ブラジルジャディウソン: 1
イバンチェビッチ: 1
ホームシック: 1
ビタウ: 1
アンドラジーニャ: 1
ウリダ: 1
1318: 1
智樹: 1
蛯沢: 1
斉川: 1
1524: 1
磐田原: 1
セバスティアン: 1
ダヴィ・: 1
カウエ: 1
速攻: 1
アルセウ: 1
アトチコ: 1
ユールゴーデン: 1
IFFW: 1
インデペンディエンテ・メデジン: 1
チョウ・ソンファン: 1
荒谷: 1
沖縄大学: 1
ハファエル: 1
浮き沈み: 1
1614: 1
曽田: 1
パース・グローリー: 1
クラブジオゴ: 1
39243: 1
グアラニ: 1
ジオゴ: 1
荒野: 1
ナウチコハモン: 1
フォス・ド・イグアス: 1
ハモ: 1
ブリスベン・ロアー: 1
チョ: 1
ソンジン: 1
クラブパウロン

In [10]:
# 全体の上位10の語句を除去
textValues, counts = zip(*counter.most_common(10))
for word in textValues:
    cleanedData['text'] = cleanedData['text'].str.replace(word,'')

In [11]:
# CSV出力
cleanedData.to_csv("csv/train_cleaned.csv",index = False, encoding='utf-8')